### Importing the Libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
tf.__version__ 

'2.12.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
!wget https://github.com/AvijitChowdhury/Sign-Language-To-Text-Conversion1/archive/refs/heads/main.zip



--2023-04-01 17:36:07--  https://github.com/AvijitChowdhury/Sign-Language-To-Text-Conversion1/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/AvijitChowdhury/Sign-Language-To-Text-Conversion1/zip/refs/heads/main [following]
--2023-04-01 17:36:07--  https://codeload.github.com/AvijitChowdhury/Sign-Language-To-Text-Conversion1/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [             <=>    ] 421.79M  26.4MB/s    in 16s     

2023-04-01 17:36:24 (25.8 MB/s) - ‘main.zip’ saved [442279055]



In [ ]:
!unzip main.zip

Streaming output truncated to the last 5000 lines.
  inflating: Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/P/315.jpg  
  inflating: Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/P/316.jpg  
  inflating: Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/P/317.jpg  
  inflating: Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/P/318.jpg  
  inflating: Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/P/32.jpg  
  inflating: Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/P/320.jpg  
  inflating: Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/P/321.jpg  
  inflating: Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/P/322.jpg  
  inflating: Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/P/323.jpg  
  inflating: Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/P/324.jpg  
  inflating: Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/P/325.jpg  
  inflating: Si

### Creating the Training set

In [ ]:
!mkdir trainingData

In [ ]:
!mkdir testingData

In [ ]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/S' 'trainingData'

In [ ]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/M' 'trainingData'

In [ ]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/N' 'trainingData'

In [ ]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/testingData/S' 'testingData'

In [ ]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/testingData/M' 'testingData'

In [ ]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/testingData/N' 'testingData'

In [ ]:
training_set = train_datagen.flow_from_directory('trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 1450 images belonging to 3 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 482 images belonging to 3 classes.


# Part 2: Building CNN


# Initializing CNN

In [ ]:
classifier = tf.keras.models.Sequential()

#Step 1: Convolution

In [ ]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#Step 2:  Pooling

In [ ]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

# Adding a second convolution layer

In [ ]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

# Step 3: Flattering

In [ ]:
classifier.add(tf.keras.layers.Flatten())

# Step 4: Full Connection

In [30]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=32, activation='relu')) # softmax for more than 2
classifier.add(tf.keras.layers.Dense(units=16, activation='relu')) # softmax for more than 2
classifier.add(tf.keras.layers.Dense(units=3, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [31]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [32]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 128)               4

In [33]:
classifier

In [34]:
classifier.fit(training_set,
                  epochs = 30,
                  validation_data = test_set)

Epoch 1/30
145/145 [==============================] - 49s 303ms/step - loss: 1.1005 - accuracy: 0.3234 - val_loss: 1.0982 - val_accuracy: 0.3423
Epoch 2/30
145/145 [==============================] - 41s 284ms/step - loss: 1.0994 - accuracy: 0.3269 - val_loss: 1.0986 - val_accuracy: 0.3382
Epoch 3/30
145/145 [==============================] - 42s 290ms/step - loss: 1.0994 - accuracy: 0.3352 - val_loss: 1.0982 - val_accuracy: 0.3423
Epoch 4/30
145/145 [==============================] - 41s 285ms/step - loss: 1.0990 - accuracy: 0.3255 - val_loss: 1.0983 - val_accuracy: 0.3423
Epoch 5/30
145/145 [==============================] - 44s 301ms/step - loss: 1.0988 - accuracy: 0.3324 - val_loss: 1.0982 - val_accuracy: 0.3423
Epoch 6/30
145/145 [==============================] - 43s 296ms/step - loss: 1.0993 - accuracy: 0.3097 - val_loss: 1.0983 - val_accuracy: 0.3382
Epoch 7/30
145/145 [==============================] - 44s 301ms/step - loss: 1.0987 - accuracy: 0.3269 - val_loss: 1.0982 - val_ac

#### Saving the Model

In [35]:
model_json = classifier.to_json()
with open("model_bw-smn.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_bw-smn.h5')
print('Weights saved')

Model Saved
Weights saved
